In [ ]:
import math
import sys

import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline

from matplotlib import rcParams
import matplotlib.font_manager

plt.rcParams.update(
    {
        "figure.figsize": (5, 3),
        "figure.max_open_warning": False,
        "text.usetex": True,
        "font.family": "DejaVu Sans",
    }
)

# Simulation

In [ ]:
from crazyflie_description_py.parameters import N_BUFFER, FS
from utils.simulation import WIDEBAND_FILE, create_wideband_signal

mic_idx = 1
distances = np.arange(10, 60, step=1)
frequencies = np.fft.rfftfreq(N_BUFFER, 1 / FS)
min_freq = 100
max_freq = 5000
min_dist = 1
max_dist = 50
freq_start = int(min_freq / max(frequencies) * len(frequencies))
freq_end = int(max_freq / max(frequencies) * len(frequencies))
dist_start = int(min_dist / max(distances) * len(distances))
dist_end = int(max_dist / max(distances) * len(distances))

dist = distances[dist_start:dist_end]
freq = frequencies[freq_start:freq_end]

signal = create_wideband_signal(freq)

In [ ]:
## implement and test LMCV beamformer
from utils.simulation import generate_room, get_buffers
from utils.plotting_tools import save_fig
from audio_stack.beam_former import BeamFormer
from pyroomacoustics.soundsource import SoundSource

distance_cm = 500 # in the middle of the room
room = generate_room(distance_cm=distance_cm, source=False)

mic_center = np.mean(room.mic_array.R, axis=1) # d x mics
mics = room.mic_array.R - mic_center[:, None] # 2, 4
beamformer = BeamFormer(mic_positions=mics.T)

# add a sound source at given angle from mics
source_angle_deg = 30
source = mic_center + 1 * np.r_[np.cos(source_angle_deg/180*np.pi),
                                np.sin(source_angle_deg/180*np.pi)]
buzzer = SoundSource(mic_center, signal=signal)
virtual_source = SoundSource(source, signal=signal*0.5)

name=f"double{source_angle_deg}"
room.add_soundsource(virtual_source)
room.add_soundsource(buzzer)

#name="single"
#room.add_soundsource(virtual_source)
    
fig=plt.figure()
room.plot()
plt.xlabel("x [m]")
plt.ylabel("y [m]")
plt.title("full room")


In [ ]:
signals_f = get_buffers(room, signal, n_times=2) # 1 x 4 x n_freqs
signals_f = signals_f[0, :, :].T
nnz = np.where(np.sum(np.abs(signals_f), axis=1) > 10.0)[0]

plt.figure()
for i in range(signals_f.shape[1]):
    plt.plot(frequencies, np.abs(signals_f[:, i]))
    plt.plot(frequencies[nnz], np.abs(signals_f[nnz, i]), color="k", ls=":")

fig, ax = plt.subplots()
room.plot(ax=ax)

buzzer_constraints = []
# IMPORTANT: can only add 3 constraints because then we have a total of 
# 4 (number of mics). 
for i, mic in enumerate(mics.T[:2]):
    theta_mic = np.arctan2(mic[1], mic[0])
    buzzer_constraints.append((theta_mic, 0))
    ray = 2 * np.r_[np.cos(theta_mic), np.sin(theta_mic)]
    ax.plot([mic_center[0], mic_center[0] + ray[0]], 
            [mic_center[1], mic_center[1] + ray[1]], ls=":", label="buzzer direction")
    
prop_constraints = []
for i, mic in enumerate(mics.T[:2]):
    theta_prop = np.arctan2(mic[1], mic[0]) + np.pi / 4
    prop_constraints.append((theta_prop, 0))
    ray = 2 * np.r_[np.cos(theta_prop), np.sin(theta_prop)]
    ax.plot([mic_center[0], mic_center[0] + ray[0]], 
            [mic_center[1], mic_center[1] + ray[1]], ls=":", label="prop direction")
ax.plot([mic_center[0], source[0]], 
        [mic_center[1], source[1]], color='k', ls=":", label="source direction")
ax.legend()
ax.set_xlabel("x [m]")
ax.set_ylabel("y [m]")
mins = np.min(np.c_[room.mic_array.R,source], axis=1)*0.95
maxs = np.max(np.c_[room.mic_array.R,source], axis=1)*1.05
ax.axis("equal")
ax.set_xlim(mins[0], maxs[0])
ax.set_ylim(mins[1], maxs[1])
ax.set_title("zoom")
save_fig(fig, f"plots/theory/{name}source_setup_zoom.pdf")

In [ ]:
# combine mics signals with beamforming. 
R = beamformer.get_correlation(signals_f[nnz])
spec_lcmv = beamformer.get_lcmv_spectrum(R, frequencies_hz=frequencies[nnz], 
                                         extra_constraints=buzzer_constraints,
                                         cancel_centre=False)
spec_cent = beamformer.get_lcmv_spectrum(R, frequencies_hz=frequencies[nnz], 
                                         extra_constraints=[], 
                                         cancel_centre=True)
spec_prop = beamformer.get_lcmv_spectrum(R, frequencies_hz=frequencies[nnz], 
                                         extra_constraints=prop_constraints, 
                                         cancel_centre=True)
spec_mvdr = beamformer.get_mvdr_spectrum(R, frequencies_hz=frequencies[nnz])
spec_das = beamformer.get_das_spectrum(R, frequencies_hz=frequencies[nnz])
spec_phat = beamformer.get_das_spectrum(R, frequencies_hz=frequencies[nnz], phat=True)

In [ ]:
fig, ax = plt.subplots()
im = ax.pcolormesh(beamformer.theta_scan_deg, 
                   frequencies[nnz], 
                   np.log10(spec_cent))
ax.axvline(source_angle_deg, color='white', ls=":")
plt.colorbar(im)
for c in buzzer_constraints:
    if c[0] < 0:
        zero_angle = c[0] * 180 / np.pi + 360
    else:
        zero_angle = c[0] * 180 / np.pi
    ax.axvline(zero_angle, color='black', ls=":")
ax.set_xlabel("angles")
ax.set_ylabel("frequencies")

In [ ]:
from utils.inference import get_1d_spectrum, get_angle_distribution

angles = beamformer.theta_scan_deg
dict_ = {"LCMV": spec_lcmv, 
         "LCMV centre": spec_cent,
         "LCMV prop": spec_prop,
         "MVDR": spec_mvdr, 
         "DAS": spec_das,
         "GCC-PHAT": spec_phat}
fig, axs = plt.subplots(1, len(dict_), sharey=True)
for i, (title, spec) in enumerate(dict_.items()):
    #axs[i].semilogy(angles, np.sum(spec, axis=0))
    
    vals = get_1d_spectrum(spec)
    
    axs[i].plot(angles, vals)
    axs[i].axvline(source_angle_deg, color="k", ls=":", label="source direction")
    axs[i].set_title(title)
    if title == "LCMV":
        for j, (theta, __) in enumerate(buzzer_constraints):
            if theta < 0:
                theta += 2 * np.pi
            axs[i].axvline(theta * 180 / np.pi, color=f"C{j}", ls=":", label="canceled direction")
    axs[i].set_xlabel("angle [deg]")
    axs[i].legend(loc="lower right")
axs[0].set_ylabel("normalized spectrum")
fig.set_size_inches(15, 5)
save_fig(fig, f"plots/theory/{name}source_results.pdf")

In [ ]:
fig, ax = plt.subplots() 
source_angles_deg = np.arange(91, step=30)
distance_cm = 500
for j, source_angle_deg in enumerate(source_angles_deg):
    room = generate_room(distance_cm=distance_cm, source=False)
    
    mic_center = np.mean(room.mic_array.R, axis=1) # d x mics
    # add a sound source at given angle from mics
    source = mic_center + 1 * np.r_[np.cos(source_angle_deg/180*np.pi),
                                    np.sin(source_angle_deg/180*np.pi)]
    buzzer = SoundSource(mic_center, signal=signal)
    virtual_source = SoundSource(source, signal=signal*0.7)
    room.add_soundsource(virtual_source)
    room.add_soundsource(buzzer)
    
    signals_f = get_buffers(room, signal, n_times=2) # 1 x 4 x n_freqs
    signals_f = signals_f[0, :, :].T
    
    angles, probs = get_angle_distribution(signals_f[nnz], frequencies[nnz], mics)
    ax.plot(angles, probs, color=f"C{j}", label=f"source at {source_angle_deg}$^\circ$")
    ax.axvline(source_angle_deg, color=f"C{j}", ls=":")
ax.set_xlabel("angle [deg]")
ax.set_ylabel("probability [-]")
ax.legend()

In [ ]:
from scipy.io import wavfile 
fs, prop_signal = wavfile.read("./2021_10_07_stepper_new_f_export_motors_binsel5_noprops_sweep_new_10.wav")
time_range=[7, 8]
prop_signal = prop_signal[int(time_range[0]*fs):int(time_range[1]*fs)] 
prop_signal = wavfile.write("./prop_signal.wav", fs, prop_signal)

In [ ]:
def add_propeller_signals(room, amplitude_ratio=0.1):
    """
    snr=0.1 means noise as 0.1 of average signal amplitude
    """
    import scipy
    from utils.signals import get_power
    mic_center = np.mean(room.mic_array.R, axis=1) # d x mics
    mics = room.mic_array.R - mic_center[:, None] # 2, 4
    
    radius_m = np.mean(np.linalg.norm(mics, axis=0))
    new_radius_m = 0.05  # 5 cm
    a = 45 * np.pi / 180
    R = np.c_[[np.cos(a), np.sin(a)], [-np.sin(a), np.cos(a)]]
    propellers = R @ mics
    propellers *= new_radius_m / radius_m
    propellers += mic_center[:, None]
    
    # Your new sampling rate
    new_rate = room.fs
    sampling_rate, data = wavfile.read("./prop_signal.wav")
    number_of_samples = round(len(data) * float(new_rate) / sampling_rate)
    data = scipy.signal.resample(data, number_of_samples)
    
    # make sure the sound and propeller power is the same. 
    signal_power = get_power(room.sources[0].signal, dB=False)
    noise_power = get_power(data, dB=False)
    data *= np.sqrt(signal_power/noise_power)
    
    # adjust amplitude
    data *= amplitude_ratio
    
    for i, p in enumerate(propellers.T):
        soundsource = SoundSource(p, signal=data[i*100:])
        room.add_soundsource(soundsource)
        
add_propeller_signals(room)

In [ ]:
## study for a close wall
from utils.estimators import DistanceEstimator
from utils.signals import get_power
from utils.geometry import Context

dist_here = np.arange(10, 60, step=10) #dist[::5]

#azimuth_deg = 132 # normal value
azimuth_deg = 120 # normal value

# determine signal frequency bins
room = generate_room(distance_cm=50, azimuth_deg=azimuth_deg)
signals_f = get_buffers(room, signal*100, n_times=2) # 1 x 4 x n_freqs
signals_f = signals_f[0, :, :].T
#nnz = np.where(np.sum(np.abs(signals_f), axis=1) > 100.0)[0]
nnz = np.where((frequencies > 1000) & (frequencies < 4500))[0]

context = Context.get_crazyflie_setup(dim=2)
#mic_center = np.mean(room.mic_array.R, axis=1) # d x mics
#mics = room.mic_array.R - mic_center[:, None] # 2, 4
beamformer = BeamFormer(mic_positions=context.mics)
plt.figure()
for m in context.mics:
    plt.scatter(*m)
plt.axis("equal")

plt.figure()
plt.plot(frequencies[nnz], np.abs(signals_f[nnz,0]))

fig1, ax1 = plt.subplots()
fig2, ax2 = plt.subplots()
fig3, ax3 = plt.subplots()
fig4, ax4 = plt.subplots()
cmap = plt.get_cmap("viridis", lut=len(dist_here))
for j, distance_cm in enumerate(dist_here):
    print(f"distance {j+1}/{len(dist_here)}")
    room = generate_room(distance_cm=distance_cm, azimuth_deg=azimuth_deg, source=False)
    
    mic_center = np.mean(room.mic_array.R, axis=1) # d x mics
    buzzer = SoundSource(mic_center, signal=signal)
    room.add_soundsource(buzzer)
    
    add_propeller_signals(room, 0.6)
    signal_power = get_power(room.sources[0].signal)
    noise_power = get_power(room.sources[1].signal)
    #print(signal_power, noise_power)
    
    signals_f = get_buffers(room, signal*100, n_times=2) # 1 x 4 x n_freqs
    signals_f = signals_f[0, :, :].T
    
    # get angle estimate with beamforming.
    R = beamformer.get_correlation(signals_f[nnz])
    spec = beamformer.get_lcmv_spectrum(R, frequencies_hz=frequencies[nnz], 
                                        #extra_constraints=prop_constraints,
                                        cancel_centre=True)
    
    vals = get_1d_spectrum(spec)
    
    ax1.plot(beamformer.theta_scan_deg, vals, color=cmap(j), label=f"{distance_cm:.0f}cm")
    ax2.plot(frequencies[nnz], np.abs(signals_f[nnz, 0]), color=cmap(j), label=f"{distance_cm:.0f}cm")
    
    distance_estimator = DistanceEstimator()
    distance_estimator.add_distributions(signals_f[nnz], frequencies[nnz], azimuth_deg=azimuth_deg)
    
    d_prob, prob = distance_estimator.get_distance_distribution()
    ax3.plot(d_prob, prob, color=cmap(j), label=f"{distance_cm:.0f}cm")
    
    a_prob, prob = distance_estimator.get_angle_distribution(distance_estimate_cm=distance_cm)
    ax4.plot(a_prob, prob, color=cmap(j), label=f"{distance_cm:.0f}cm")
    
ax1.legend()
ax2.legend()

In [ ]:
## study of the matrix after beamforming
from utils.geometry import Context
context = Context.get_platform_setup()
beamformer = BeamFormer(mic_positions=context.mics)

cmap = plt.get_cmap("viridis", lut=len(dist))

df_matrix_beamformed = np.empty((len(frequencies[nnz]), len(dist)))
for j, distance_cm in enumerate(dist):
    room = generate_room(distance_cm=distance_cm)
    
    signals_f = get_buffers(room, signal, n_times=2) # 1 x 4 x n_freqs
    signals_f = signals_f[0, :, :].T
    
    # combine mics signals with beamforming. 
    R = beamformer.get_correlation(signals_f[nnz])
    coeffs = beamformer.beamform_lcmv(R, np.pi/2, frequencies_hz=frequencies[nnz], cancel_centre=True)
    slice_total = np.sum(np.multiply(coeffs, signals_f[nnz]), axis=1)
    df_matrix_beamformed[:, j] = np.abs(slice_total) ** 2
    
fig, ax2 = plt.subplots()
ax2.pcolorfast(dist, freq, np.log10(df_matrix_beamformed[freq_start:freq_end, :]))

# Real data

In [ ]:
from generate_flying_results import combine_stepper_df
stepper_df = pd.read_pickle("../datasets/2021_07_08_stepper_fast/all_data.pkl")
data_df = combine_stepper_df(stepper_df, motors="all45000", bin_selection=5, average=False)
print(data_df)

In [ ]:
from crazyflie_demo.wall_detection import WallDetection

np.random.seed(1) # for particle filter

freqs_all = data_df.frequencies_matrix[0, :]
nnz = freqs_all > 0

context = Context.get_platform_setup()

n_positions = data_df.positions.shape[0]

alpha = 0.3

angle_estimates = []
fig, ax = plt.subplots()
fig.set_size_inches(10, 5)
chosen = np.arange(n_positions)[:30]
print(chosen)
cmap = plt.get_cmap('viridis', lut=len(chosen))
distances_cm = -data_df.positions[chosen, 1]

WallDetection.BEAMFORM = True
wall_detection = WallDetection(python_only=True, estimator="histogram")

for i_col, i in enumerate(chosen):
    print(f"{i_col+1}/{len(chosen)}")
    position_cm = data_df.positions[i, :3]
    signals_f = data_df.stft[i]
    yaw_deg = data_df.positions[i, 3]
    
    return_dict = wall_detection.listener_callback_offline(signals_f[:, nnz].T, freqs_all[nnz], position_cm, yaw_deg, timestamp=i_col)
    #angles = return_dict["angle_static"]
    #probs = return_dict["prob_angle_static"]
    
    angles = return_dict["angle_moving"]
    probs = return_dict["prob_angle_moving"]
    #angles, probs = get_angle_distribution(
    #    signals_f[:, nnz].T, freqs_all[nnz], context.mics.T
    #)
    ax.plot(angles, probs, color=cmap(i_col), label=distances_cm[i_col])
    angle_estimates.append(angles[np.argmax(probs)])
ax.legend()
ax.set_xlabel("angle [deg]")
ax.set_ylabel("probability [-]")
    
plt.figure()
plt.plot(distances_cm, angle_estimates)
plt.xlabel("distance [cm]")
plt.ylabel("angle estimate [deg]")

In [ ]:
from scipy.stats import norm

states = np.arange(10)

posterior = norm.pdf(states, loc=1, scale=1.0)
plt.figure()
plt.plot(posterior)
sigma_d = 2
norm = np.exp(-(states[:, None] - states[None, :])**2 / sigma_d)
print(norm.shape)
plt.figure()
plt.matshow(norm)

prior = np.sum(norm * posterior[:, None], axis=0)
prior /= np.sum(prior)
plt.figure()
plt.plot(prior)